In [103]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import scipy.stats as st 
from tqdm import tqdm 
import os
## Change directory to your working directory
os.chdir('D:\Shashi Katteri\Desktop\data sets to work on\Amazon Employee Access\Dataset_10')

In [104]:
df=pd.read_csv('Amazon_employee_access.csv')
df.head()

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE,target
0,39353,85475,117961,118300,123472,117905,117906,290919,117908,1
1,17183,1540,117961,118343,123125,118536,118536,308574,118539,1
2,36724,14457,118219,118220,117884,117879,267952,19721,117880,1
3,36135,5396,117961,118343,119993,118321,240983,290919,118322,1
4,42680,5905,117929,117930,119569,119323,123932,19793,119325,1


In [105]:
df1=df.astype(str)
df1['target']=df1['target'].astype(int)

In [106]:
#df1['ROLE_ROLLUP_12']=df1['ROLE_ROLLUP_1']+df1['ROLE_ROLLUP_2']
df1=df1.drop(['ROLE_TITLE','ROLE_FAMILY'],axis=1)


In [107]:
df1['target']=df1['target'].map({1:0,0:1})

In [108]:
df1['ROLLUP_12']=df1['ROLE_ROLLUP_1']+df1['ROLE_ROLLUP_2']
df1=df1.drop(['ROLE_ROLLUP_1','ROLE_ROLLUP_2'],axis=1)

In [109]:
df1.head()

,RESOURCE,MGR_ID,ROLE_DEPTNAME,ROLE_FAMILY_DESC,ROLE_CODE,target,ROLLUP_12
0,39353,85475,123472,117906,117908,0,117961118300
1,17183,1540,123125,118536,118539,0,117961118343
2,36724,14457,117884,267952,117880,0,118219118220
3,36135,5396,119993,240983,118322,0,117961118343
4,42680,5905,119569,123932,119325,0,117929117930


# Combinations with brute force 


In [110]:
from itertools import combinations

cols=list(df1.columns)
cols.remove('target')
d=df1.copy()

for j in combinations(cols,2):
    d[j[0]+'_'+j[1]]=df1[j[0]]+df1[j[1]]
    
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32769 entries, 0 to 32768
Data columns (total 22 columns):
RESOURCE                          32769 non-null object
MGR_ID                            32769 non-null object
ROLE_DEPTNAME                     32769 non-null object
ROLE_FAMILY_DESC                  32769 non-null object
ROLE_CODE                         32769 non-null object
target                            32769 non-null int64
ROLLUP_12                         32769 non-null object
RESOURCE_MGR_ID                   32769 non-null object
RESOURCE_ROLE_DEPTNAME            32769 non-null object
RESOURCE_ROLE_FAMILY_DESC         32769 non-null object
RESOURCE_ROLE_CODE                32769 non-null object
RESOURCE_ROLLUP_12                32769 non-null object
MGR_ID_ROLE_DEPTNAME              32769 non-null object
MGR_ID_ROLE_FAMILY_DESC           32769 non-null object
MGR_ID_ROLE_CODE                  32769 non-null object
MGR_ID_ROLLUP_12                  32769 non-null obj

In [130]:
def AST_encoding(train,test,columns,target,weight):
    tr=train.copy()
    v=test.copy()
    columns=list(columns)
   
    gm=tr[target].mean()
    for col in columns:
        gb=tr.groupby(col)[target].agg(['count','mean'])
        count=gb['count']
        mean=gb['mean']
        gb['smooth']=(count*mean + weight*gm)/(count+weight)
        tr[col+'_enc']=tr[col].map(gb['smooth'])
        
        v[col+'_enc']=v[col].map(gb['smooth'])
        v[col+'_enc']=v[col+'_enc'].fillna(gm)
    v=v.drop(columns,axis=1)
    tr=tr.drop(columns,axis=1)
    return tr,v

In [139]:
def aucroc(mod,x,y):
    pred=mod.predict(x)
    prob=mod.predict_proba(x)[:,1]
    auc=roc_auc_score(y,prob)
    acc=balanced_accuracy_score(y,pred)
    f1=f1_score(y,pred)
    return auc,acc,f1
    #print('AUC_ROC= ',auc)
    #print('Accuracy= ',acc)
    #print('confusion matrix:\n',confusion_matrix(y,pred))

In [113]:
from lightgbm import LGBMClassifier
from sklearn.metrics import *
from sklearn.model_selection import train_test_split,RandomizedSearchCV,StratifiedKFold
from sklearn.linear_model import LogisticRegression



In [114]:
rem=['RESOURCE_ROLE_CODE', 'ROLE_FAMILY_DESC_ROLLUP_12', 'MGR_ID', 'ROLE_DEPTNAME_ROLE_FAMILY_DESC',
     'MGR_ID_ROLLUP_12', 'RESOURCE_ROLE_FAMILY_DESC', 'ROLE_DEPTNAME_ROLE_CODE', 'MGR_ID_ROLE_DEPTNAME',
     'ROLE_FAMILY_DESC', 'ROLE_CODE_ROLLUP_12', 'ROLE_DEPTNAME', 'ROLE_DEPTNAME_ROLLUP_12', 'ROLLUP_12']


In [115]:
d=d.drop(rem,axis=1)
held_back=d.sample(frac=0.2,random_state=123)
d=d.drop(held_back.index)

In [116]:
d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26215 entries, 0 to 32768
Data columns (total 9 columns):
RESOURCE                      26215 non-null object
ROLE_CODE                     26215 non-null object
target                        26215 non-null int64
RESOURCE_MGR_ID               26215 non-null object
RESOURCE_ROLE_DEPTNAME        26215 non-null object
RESOURCE_ROLLUP_12            26215 non-null object
MGR_ID_ROLE_FAMILY_DESC       26215 non-null object
MGR_ID_ROLE_CODE              26215 non-null object
ROLE_FAMILY_DESC_ROLE_CODE    26215 non-null object
dtypes: int64(1), object(8)
memory usage: 2.0+ MB


In [117]:
x=d.drop('target',axis=1)
y=d['target']
x_val=held_back.drop('target',axis=1)
y_val=held_back['target']


In [145]:
sf=StratifiedKFold(n_splits=10)
model=LogisticRegression(max_iter=10e7)
scores=pd.DataFrame(columns=['AUC-ROC','AUC-ROC Val','Accuracy','Accuracy Val','F1 score','F1 score Val'],index=range(1,11))
fold=0
fold_mod=dict()
for i,j in sf.split(x,y):
    
    fold=fold+1
    
    train=d.iloc[i]
    test=d.iloc[j]
    target='target'    
    columns=list(x.columns)
   
    t,v=AST_encoding(train,test,columns,target,20)
    tr,val=AST_encoding(train,held_back,columns,target,20)
    
    xtr=t.drop(target,axis=1)
    xts=v.drop(target,axis=1)
    ytr=t[target]
    yts=v[target]
    x_val=val.drop('target',axis=1)
    y_val=val['target']
    del tr
    
    mod=model.fit(xtr,ytr)
    fold_mod[fold]=mod
    auc,acc,f1=aucroc(mod,xts,yts)
    auc_val,acc_val,f1_val=aucroc(mod,x_val,y_val)
    
    scores.loc[fold,['AUC-ROC','Accuracy','F1 score']]= auc,acc,f1
    scores.loc[fold,['AUC-ROC Val','Accuracy Val','F1 score Val']]= auc_val,acc_val,f1_val
    
    
    
    

In [147]:
scores

,AUC-ROC,AUC-ROC Val,Accuracy,Accuracy Val,F1 score,F1 score Val
1,0.876851,0.88653,0.603532,0.575836,0.328205,0.251012
2,0.87071,0.884994,0.560688,0.573528,0.209945,0.245399
3,0.892548,0.887094,0.566299,0.566907,0.223404,0.225873
4,0.869211,0.882707,0.560283,0.563487,0.20765,0.217119
5,0.867974,0.885739,0.570217,0.565551,0.236559,0.221766
6,0.860318,0.882028,0.556155,0.570896,0.192513,0.237705
7,0.863617,0.88504,0.566762,0.569458,0.224599,0.233129
8,0.861996,0.877333,0.564036,0.566989,0.218579,0.226337
9,0.863254,0.88281,0.592215,0.567777,0.288557,0.227181
10,0.862624,0.882532,0.565548,0.573911,0.217617,0.243513


In [148]:
for i in ['AUC-ROC','Accuracy','F1 score']:
    scores[i+'_delta']=abs(scores[i]-scores[i+' Val'])
    scores[i+'delta_rank']=scores[i+'_delta'].rank(ascending=True)
scores.head()

,AUC-ROC,AUC-ROC Val,Accuracy,Accuracy Val,F1 score,F1 score Val,AUC-ROC_delta,AUC-ROCdelta_rank,Accuracy_delta,Accuracydelta_rank,F1 score_delta,F1 scoredelta_rank
1,0.876851,0.88653,0.603532,0.575836,0.328205,0.251012,0.00967916,2.0,0.0276962,10.0,0.077193,10.0
2,0.87071,0.884994,0.560688,0.573528,0.209945,0.245399,0.0142845,4.0,0.0128403,7.0,0.035454,7.0
3,0.892548,0.887094,0.566299,0.566907,0.223404,0.225873,0.0054541,1.0,0.000608494,1.0,0.00246843,1.0
4,0.869211,0.882707,0.560283,0.563487,0.20765,0.217119,0.0134956,3.0,0.00320315,4.0,0.00946872,4.0
5,0.867974,0.885739,0.570217,0.565551,0.236559,0.221766,0.0177649,6.0,0.00466646,5.0,0.0147932,5.0


In [149]:
scores['PowerRank']=scores['AUC-ROCdelta_rank']*scores['Accuracydelta_rank']*scores['F1 scoredelta_rank']
scores

,AUC-ROC,AUC-ROC Val,Accuracy,Accuracy Val,F1 score,F1 score Val,AUC-ROC_delta,AUC-ROCdelta_rank,Accuracy_delta,Accuracydelta_rank,F1 score_delta,F1 scoredelta_rank,PowerRank
1,0.876851,0.88653,0.603532,0.575836,0.328205,0.251012,0.00967916,2.0,0.0276962,10.0,0.077193,10.0,200.0
2,0.87071,0.884994,0.560688,0.573528,0.209945,0.245399,0.0142845,4.0,0.0128403,7.0,0.035454,7.0,196.0
3,0.892548,0.887094,0.566299,0.566907,0.223404,0.225873,0.0054541,1.0,0.000608494,1.0,0.00246843,1.0,1.0
4,0.869211,0.882707,0.560283,0.563487,0.20765,0.217119,0.0134956,3.0,0.00320315,4.0,0.00946872,4.0,48.0
5,0.867974,0.885739,0.570217,0.565551,0.236559,0.221766,0.0177649,6.0,0.00466646,5.0,0.0147932,5.0,150.0
6,0.860318,0.882028,0.556155,0.570896,0.192513,0.237705,0.0217092,10.0,0.0147408,8.0,0.0451915,8.0,640.0
7,0.863617,0.88504,0.566762,0.569458,0.224599,0.233129,0.0214235,9.0,0.00269594,2.0,0.0085299,3.0,54.0
8,0.861996,0.877333,0.564036,0.566989,0.218579,0.226337,0.0153366,5.0,0.00295235,3.0,0.00775821,2.0,30.0
9,0.863254,0.88281,0.592215,0.567777,0.288557,0.227181,0.0195566,7.0,0.0244379,9.0,0.0613767,9.0,567.0
10,0.862624,0.882532,0.565548,0.573911,0.217617,0.243513,0.0199082,8.0,0.00836292,6.0,0.0258964,6.0,288.0


In [151]:
scores.apply(np.mean)

AUC-ROC                 0.868910
AUC-ROC Val             0.883681
Accuracy                0.570574
Accuracy Val            0.569434
F1 score                0.234763
F1 score Val            0.232903
AUC-ROC_delta           0.015861
AUC-ROCdelta_rank       5.500000
Accuracy_delta          0.010220
Accuracydelta_rank      5.500000
F1 score_delta          0.028813
F1 scoredelta_rank      5.500000
PowerRank             217.400000
dtype: float64